In [119]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [120]:
import os, glob
import matplotlib.pyplot as plt
import numpy as np
import pickle
import nibabel as nb
import matplotlib.image as mpimg
import cv2
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.utils import normalize
from tensorflow.keras.utils import to_categorical
from tqdm.notebook import tqdm 
from sys import getsizeof
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import MeanIoU

import nibabel as nib

In [121]:
# Define constants

# Path variables
model1_path = '/content/drive/Shareddrives/Banana Leaf Development/Data/Couinaud_Annotation_Data/model_notebooks/CompleteDataSet/2-Models:0-4 and 0,5-8/Model1'
model2_path = '/content/drive/Shareddrives/Banana Leaf Development/Data/Couinaud_Annotation_Data/model_notebooks/CompleteDataSet/2-Models:0-4 and 0,5-8/Model2'

data_dir = '/content/drive/Shareddrives/Banana Leaf Development/Data/Couinaud_Annotation_Data/'
data_dir_test_image = os.path.join(data_dir, 'complete_dataset/testing/img:-1000:1000/img/')
data_dir_test_mask = os.path.join(data_dir, 'complete_dataset/testing/couinaud/img/')

# Training variables
SEED = 7
BATCH_SIZE_TRAIN = 10
BATCH_SIZE_TEST = 10
BATCH_SIZE_VALIDATE = 10
NUM_TRAIN = 9288
NUM_TEST = 1042
NUM_VALIDATE = 1207

N_CLASSES = 9


# Image variables
IMAGE_HEIGHT = int(512/4)
IMAGE_WIDTH = int(512/4)
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)


#Original Image size
OG_IMAGE_HEIGHT = 512
OG_IMAGE_WIDTH = 512
OG_IMG_SIZE = (OG_IMAGE_HEIGHT, OG_IMAGE_WIDTH)

NUM_EPOCH = 100



Load Testing Data

In [122]:
#Load Testing Images
test_images = []
scaled_test_images = []

for index, img_path in tqdm(enumerate(sorted(glob.iglob(data_dir_test_image+ '*.png')))):
  substring = img_path[-5:]

  img = cv2.imread(img_path, 0)       
  test_images.append(img)
  scaled_test_images.append(cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_LINEAR))

#Convert list to array for machine learning processing        
test_images = np.array(test_images)
scaled_test_images = np.array(scaled_test_images)

print('##### TESTING IMAGES #####')
print(test_images.shape)
print(scaled_test_images.shape)
print(type(test_images[0,0,0]))


##### TESTING IMAGES #####
(1042, 512, 512)
(1042, 128, 128)
<class 'numpy.uint8'>


In [123]:
#Load Testing Masks for Model 1
#test_masks = [] 
scaled_test_masks = []

for index, img_path in tqdm(enumerate(sorted(glob.iglob(data_dir_test_mask + '*.tif')))):
  substring = img_path[-5:]

  img = cv2.imread(img_path, 0)       
  #test_masks.append(img)
  other_mask_indices = img > 4
  img[other_mask_indices] = 5
  scaled_test_masks.append(cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_LINEAR))

#Convert list to array for machine learning processing      
#test_masks = np.array(test_masks)
scaled_test_masks1 = np.array(scaled_test_masks)

print('##### TESTING MASKS #####') 
#print(test_masks.shape)
print(scaled_test_masks1.shape)
#print(type(test_masks[0,0,0]))


##### TESTING MASKS #####
(1042, 128, 128)


In [124]:
#Load Testing Masks for Model 2
#test_masks = [] 
scaled_test_masks = []

for index, img_path in tqdm(enumerate(sorted(glob.iglob(data_dir_test_mask + '*.tif')))):
  substring = img_path[-5:]

  img = cv2.imread(img_path, 0)       
  #test_masks.append(img)
  other_mask_indices = (img > 0) & (img <= 4)
  img[other_mask_indices] = 9 #bc originally there are 0-8 categories
  indices_all_but_0val = img > 0
  img[indices_all_but_0val] = img[indices_all_but_0val] - 4
  scaled_test_masks.append(cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_LINEAR))

#Convert list to array for machine learning processing      
#test_masks = np.array(test_masks)
scaled_test_masks2 = np.array(scaled_test_masks)

print('##### TESTING MASKS #####') 
#print(test_masks.shape)
print(scaled_test_masks2.shape)
#print(type(test_masks[0,0,0]))


##### TESTING MASKS #####
(1042, 128, 128)


In [125]:
#Test masks but no encoding for calculating IOU
test_masks_no_encode = [] 
scaled_test_masks_no_encode = []
for index, img_path in enumerate(sorted(glob.iglob(data_dir_test_mask + '*.tif'))):
  substring = img_path[-5:]

  img = cv2.imread(img_path, 0) 
  test_masks_no_encode.append(img)
  scaled_test_masks_no_encode.append(cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_LINEAR))

#Convert list to array for machine learning processing      
test_masks_no_encode = np.array(test_masks_no_encode)
scaled_test_masks_no_encode = np.array(scaled_test_masks_no_encode)

test_masks_no_encode = np.expand_dims(test_masks_no_encode, axis=3)
scaled_test_masks_no_encode = np.expand_dims(scaled_test_masks_no_encode, axis=3)

print(test_masks_no_encode.shape)
print(scaled_test_masks_no_encode.shape)

(1042, 512, 512, 1)
(1042, 128, 128, 1)


Import Models

In [126]:
#Load Models
model1 = load_model(model1_path)
model2 = load_model(model2_path)

Prediting Masks

In [127]:
mask_pred1=model1.predict(scaled_test_images)
mask_pred_argmax1=np.argmax(mask_pred1, axis=3) #Convert the probability into a max value within the area \

In [128]:
mask_pred2=model2.predict(scaled_test_images)
mask_pred_argmax2=np.argmax(mask_pred2, axis=3) #Convert the probability into a max value within the area \

In [129]:
#grab the 0th layer from one of the two mask prediction models
# and expand dimensions to have a 4th 
zero_from_mask1 = np.expand_dims(mask_pred1[:,:,:,0], axis=3)
print(zero_from_mask1.shape)

(1042, 128, 128, 1)


In [130]:
#Pull the predicting masks for class 1-4 from Model 1
class_1_to_4 = mask_pred1[:,:,:,1:5]
print(class_1_to_4.shape)

(1042, 128, 128, 4)


In [131]:
#Pull the predicting masks for class 5-8 from Model 2
class_5_to_8 = mask_pred2[:,:,:,1:5]
print(class_5_to_8.shape)

(1042, 128, 128, 4)


Merge the masks from the 2 models

In [132]:
stacked_class_predictions = np.concatenate((zero_from_mask1, class_1_to_4, class_5_to_8), axis =3)
print(stacked_class_predictions.shape)

models_mask_pred_argmax = np.argmax(stacked_class_predictions, axis = 3)
print(models_mask_pred_argmax.shape)

(1042, 128, 128, 9)
(1042, 128, 128)


Evaluate Model with IoU

In [133]:
IOU = MeanIoU(num_classes=N_CLASSES)  
IOU.update_state(scaled_test_masks_no_encode[:,:,:,0], models_mask_pred_argmax)
print("Mean IoU =", IOU.result().numpy()) #Prev0.61207

Mean IoU = 0.6112461


In [134]:
def calculateClassIOU(cm, num_classes=9):
  #: IOU = true_positive / (true_positive + false_positive + false_negative)
  #cm: confusion matrix where rows are actual and cols are predicted 
  
  for i in range(0,num_classes):
    trueP = cm[i,i]
    falseP = 0
    falseN = 0

    for col in range(0,num_classes):
      if (i != col):
        falseP = falseP + cm[i,col]

    for row in range(0,num_classes):
      if (i != row):
        falseN = falseN + cm[row,i]

    IoU = trueP / (trueP + falseP + falseN)
    print("IoU for class,",i, " is: ", IoU)

In [135]:
#Get confusion Matrix
values = np.array(IOU.get_weights()).reshape(N_CLASSES, N_CLASSES)
print(values)

[[1.6100205e+07 9.1450000e+03 7.6940000e+03 5.6270000e+03 3.9320000e+03
  4.3120000e+03 1.3790000e+03 2.6530000e+03 1.9220000e+03]
 [5.9740000e+03 3.3646000e+04 3.2530000e+03 5.8400000e+02 1.0680000e+03
  1.0560000e+03 4.4000000e+01 1.4060000e+03 2.1040000e+03]
 [6.9040000e+03 1.9670000e+03 9.4150000e+04 8.4630000e+03 5.8790000e+03
  3.3500000e+02 3.2200000e+02 4.4500000e+02 4.4000000e+02]
 [3.4610000e+03 5.6800000e+02 8.4260000e+03 4.3074000e+04 2.5390000e+03
  1.6530000e+03 4.7800000e+02 1.6900000e+02 2.9100000e+02]
 [2.5710000e+03 7.4100000e+02 6.3960000e+03 2.3620000e+03 6.6190000e+04
  7.1140000e+03 1.8700000e+02 8.4800000e+02 1.3832000e+04]
 [4.6890000e+03 1.4260000e+03 2.2000000e+01 3.8600000e+02 6.5790000e+03
  8.4817000e+04 7.3320000e+03 3.2050000e+03 1.9722000e+04]
 [1.9550000e+03 6.2700000e+02 1.9000000e+01 3.7900000e+02 8.5400000e+02
  1.6757000e+04 7.3104000e+04 1.5453000e+04 4.4670000e+03]
 [6.8200000e+02 1.1090000e+03 1.6000000e+01 4.3300000e+02 8.4800000e+02
  9.0000000

In [136]:
#To calculate I0U for each class...
calculateClassIOU(values, N_CLASSES)

IoU for class, 0  is:  0.9960807505577053
IoU for class, 1  is:  0.5118040766656525
IoU for class, 2  is:  0.6499288978476067
IoU for class, 3  is:  0.5447302526747098
IoU for class, 4  is:  0.4971458614991738
IoU for class, 5  is:  0.5061525791898408
IoU for class, 6  is:  0.5587452994588645
IoU for class, 7  is:  0.6558495386413361
IoU for class, 8  is:  0.5807782878618492


Print Input Image and Scaled Mask and Scaled Predicted Mask

In [139]:
def display(display_list):
  plt.figure(figsize = (15,15))
  title = ['Input Image', 'True Mask', 'Predicted Mask', 'Predicted Mask \nw/ Threshold']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]), cmap = 'Greys_r')
  plt.show()

In [ ]:
for i in range(0,50):
  print("Instance in test dataset: ", i)
  display([scaled_test_images[i], scaled_test_masks_no_encode[i], np.expand_dims(models_mask_pred_argmax[i], axis=2)])
  print("==================================================================================================================")